# Object Storage에서 Parquet 파일 읽기 예제


## 1) 필요한 라이브러리 설치


In [ ]:
pip install boto3 pandas deltalake pyarrow==15.0.2

## 2) Delta Lake에서 현재 스냅샷 로드 → 정렬 후 CSV/Parquet 저장

- `S3_ACCESS_KEY`, `S3_SECRET_KEY` 에 실제 환경 변수 값으로 수정

In [ ]:
from deltalake import DeltaTable
import pandas as pd


S3_ACCESS_KEY = "{S3_ACCESS_KEY_ID}"
S3_SECRET_KEY = "{S3_SECRET_ACCESS_KEY}"


BUCKET = "data-catalog-bucket"
PREFIX = "data-catalog-dir/user_behavior_prediction_delta/"

dt = DeltaTable(
    f"s3://{BUCKET}/{PREFIX}",
    storage_options={
        "AWS_ACCESS_KEY_ID": S3_ACCESS_KEY,        
        "AWS_SECRET_ACCESS_KEY": S3_SECRET_KEY,
        "AWS_REGION": "kr-central-2",
        "AWS_ENDPOINT_URL": "https://objectstorage.kr-central-2.kakaocloud.com",
    },
)

# Delta의 현재 스냅샷만 반영된 테이블 -> Pandas
df_all = dt.to_pandas()  # 내부적으로 _delta_log 해석해서 active 파일만 읽음
print(f"[OK] loaded delta snapshot: rows={len(df_all)}")

# ---------- 정렬 (session_id -> timestamp) ----------
# 정렬용 임시 datetime 컬럼
df_all["_ts_for_sort"] = pd.to_datetime(
    df_all["timestamp"], format="%Y-%m-%d %H:%M:%S", errors="coerce"
)

# event_id가 있으면 동률 타이브레이커로 추가
sort_cols = ["session_id", "_ts_for_sort"] + (["event_id"] if "event_id" in df_all.columns else [])

df_sorted = df_all.sort_values(
    by=sort_cols,
    kind="stable",
    na_position="last",
)

# 임시 컬럼 제거
df_sorted = df_sorted.drop(columns=["_ts_for_sort"])

# ---- 1) CSV (사람이 보기용, 로컬) ----
OUT_CSV_LOCAL = "processed_user_behavior.sorted.csv"
df_sorted.to_csv(OUT_CSV_LOCAL, index=False, encoding="utf-8")
print(f"[OK] CSV saved: {OUT_CSV_LOCAL}")

# ---- 2) Parquet (학습용, 로컬) ----
OUT_PARQUET_LOCAL = "processed_user_behavior.sorted.parquet"
df_sorted.to_parquet(OUT_PARQUET_LOCAL, index=False, engine="pyarrow", compression="snappy")
print(f"[OK] Parquet saved: {OUT_PARQUET_LOCAL}")
